## Make Treemix with Longlin, NAsns, and SAsns and NAmers
Do we see Longlin admixture into SAsns or NAmer admixture into NAsns?

In [1]:
%%writefile /mnt/solexa/mel_yang/neolithic/treemix/maketreemixfile_LLA.py
from projmodules import treemix_mkinput

pd="/mnt/solexa/mel_yang/bronzeage/data/"
#outpD="/mnt/solexa/mel_yang/neolithic/qpwaveadm/"
fh="ind_A_v13_TBH5"
subset='LLA'
setstotry=[['Tianyuan','UstIshim','Kostenki14','Vestonice16','Malta1','Longlin',
            'Daur','Hezhen','Boshan','Atayal_Coriell','Maonan','Paiwan','Rukai','Tao',
            'LiangDao','Mixe','Surui','Mbuti'], 
           ['Tianyuan','UstIshim','Kostenki14','Vestonice16','Malta1','Longlin',
            'Daur','Atayal_Coriell','LiangDao','Surui','Mbuti'], 
           ['Tianyuan','UstIshim','Kostenki14','Vestonice16','Malta1','Longlin',
            'Daur','Hezhen','Boshan','Atayal_Coriell','Maonan','Paiwan','Rukai','Tao',
            'LiangDao','Mixe','Surui','Onge','Mbuti'], 
           ['Tianyuan','UstIshim','Kostenki14','Vestonice16','Malta1','Longlin',
            'Daur','Atayal_Coriell','LiangDao','Surui','Onge','Mbuti']
           ] 
for ind,i in enumerate(setstotry):
    print ind,i
    #if ind in [2,7]: continue
    treemix_mkinput(pd,fh,subset+str(ind),setstotry[ind])    

Overwriting /mnt/solexa/mel_yang/neolithic/treemix/maketreemixfile_LLA.py


In [4]:
%%bash
pd="/mnt/solexa/mel_yang/bronzeage/data/"
fh="ind_A_v13_TBH5"
subset='LLA'
outpD="/mnt/solexa/mel_yang/neolithic/treemix/"
for i in 0 1 2 3; do 
    mv ${pd}${fh}.${subset}${i}.treemix.txt ${outpD}
done

In [12]:
%%writefile /mnt/solexa/mel_yang/neolithic/treemix/maketreemixgraphLLA.sh
pd="/mnt/solexa/mel_yang/neolithic/treemix/"
tv=""
fh="ind_A_v13_TBH5"
k="500"
myk="_k"${k}
for j in 1 2; do #`seq 0 5`; do
    subset="LLA"${j}
    gzip ${pd}${fh}.${subset}.treemix.txt
    for i in `seq 0 5`; do 
        treemix -i ${pd}${fh}.${subset}.treemix.txt.gz -root Mbuti -o ${pd}${tv}${subset}_m${i}${myk} -k ${k} -m ${i} > \
            ${pd}${tv}${subset}_m${i}${myk}.log
    done
done

##R CMD BATCH Rcode.R


Overwriting /mnt/solexa/mel_yang/neolithic/treemix/maketreemixgraphLLA.sh


In [19]:
%%bash
pd="/mnt/solexa/mel_yang/neolithic/treemix/"
tv=""
fh="ind_A_v13_TBH5"
k="500"
myk="_k"${k}
for j in 0 1 2 3; do #`seq 0 5`; do
    subset="LLA"${j}
    zcat ${pd}${fh}.${subset}.treemix.txt.gz | head -n1 | tr ' ' '\n' > ${pd}${tv}${subset}.popns.txt
done



gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [35]:
%%writefile /mnt/solexa/mel_yang/neolithic/treemix/Rcodeimage.R
args=commandArgs(TRUE)
MYPD = as.character(args[1]) ##project directory
PREFIX = as.character(args[2]) ##subset name of file
MYID = as.numeric(args[3])  ##ID Number
MYMIG = as.numeric(args[4])  ##Mig Number
setwd(MYPD)
source("plotting_funcs_MY.R")
tv=""
fh=PREFIX ##LLA
k=500
#for (id in c(0,1,2,3,4,5,6)) {
for (myid in c(MYID)) {
for (m in c(MYMIG)) {
    #pdf(paste(tv,fh,id,'.m',m,'.pdf',sep=''))
    png(paste(MYPD,'imagefiles/',tv,fh,myid,'_m',m,'_k',k,'.png',sep=''),width=580,height=580)
    par(mfrow=c(2,1))
    plot_tree(paste(MYPD,tv,fh,myid,"_m",m,"_k",k,sep=""),cex=1,plus=0.05,ybar=0.15,xmin=-0.015)
    plot_resid(paste(MYPD,tv,fh,myid,"_m",m,"_k",k,sep=""),paste(fh,myid,".popns.txt",sep=""),cex=1)
    #plot_resid(paste(tv,fh,myid,"_m",m,sep=""),paste(fh,id,".popns.txt",sep=""),cex=1,usemax=F,min=-0.001,max=0.001)
    dev.off()
    }
}

Overwriting /mnt/solexa/mel_yang/neolithic/treemix/Rcodeimage.R


In [32]:
%%writefile /mnt/solexa/mel_yang/neolithic/treemix/bootstrap.LLA1_m1.sh
##BOOTSTRAP
pd="/mnt/solexa/mel_yang/neolithic/treemix/"
tv=""
fh="ind_A_v13_TBH5"

subset="LLA1"
mig=1

reps=1000
newdir=${tv}${subset}_m${mig}_reps/

mkdir ${pd}${newdir}
myk=500
for rep in `seq 1 ${reps}`; do
    #echo ${rep}
    treemix -i ${pd}${fh}.${subset}.treemix.txt.gz -root Mbuti -bootstrap -k ${myk} -q -o ${pd}${newdir}${tv}${subset}_m${mig}_k${myk}_rep${rep} -m ${mig} > \
            ${pd}${newdir}${tv}${subset}_m${mig}_k${myk}_rep${rep}.log
done

Overwriting /mnt/solexa/mel_yang/neolithic/treemix/bootstrap.LLA1_m1.sh
